<a href="https://colab.research.google.com/github/MaximL98/Manipulated-Reality/blob/master/video_analysis/colab_notebooks/fine_tuning_pretrained_inceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Capstone Project Part II**
***This Notebook is the second from three steps, perfom fine-tuning on a pre-train model.***

In [ ]:
# Install the newest keras version
!pip install --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 32.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.


In [ ]:
# Import necessary libraries
import keras
from keras import applications
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras import backend as K
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import EarlyStopping

from PIL import Image
from pathlib import Path
import os

In [ ]:
keras.__version__

'3.4.1'

In [ ]:
# Define number of train and validation samples
train_samples = 7000
validation_samples = 3000

In [ ]:
# Dimensions of our images
img_width, img_height = 224, 224

# Set parameters
batch_size = 16
num_classes = 2
epochs = 50
activation = 'relu'
min_delta=0
patience=4
dropout=0.2
learning_rate=0.0001

In [ ]:
params={'batch_size':batch_size,
        'num_classes':num_classes,
        'epochs':epochs,
        'min_delta':min_delta,
        'patience':patience,
        'learning_rate':learning_rate,
        'dropout':dropout
}

In [ ]:
# Connect to Google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/ManipulatedReality/ImageDataset/rvf10k

inceptionv3_tuned.h5  resized_train  resized_valid  train  valid


In [ ]:
train_images_path = '/content/drive/MyDrive/ManipulatedReality/ImageDataset/rvf10k/train/'
valid_images_path = '/content/drive/MyDrive/ManipulatedReality/ImageDataset/rvf10k/valid/'

In [ ]:
# Set path to train and validation images that are stored in the drive
resized_train_images_path = '/content/drive/MyDrive/ManipulatedReality/ImageDataset/rvf10k/resized_train/'
resized_valid_images_path = '/content/drive/MyDrive/ManipulatedReality/ImageDataset/rvf10k/resized_valid/'

The images we used are not in the desired size, for that we resize them to 224x224 as our videos frame size.

In [ ]:
# Define a list of image categories
types = ['real', 'fake']
# Loop through each category
for type_ in types:
  # Get the path to the current category folder within the training images path
  # Loop through all image files in the current category
  for i, image in enumerate(os.listdir(train_images_path + type_)):
    print(f"Resizing image number {i} / 3499")
    save_path = Path(resized_train_images_path + type_ + '_resized' + '/' + image)
    # Check if the resized image already exists, skip if it does
    if save_path.is_file():
      continue
    # Open the current image from the training data path
    img = Image.open(train_images_path + type_ + '/' + image)
    # Resize the image using Lanczos resampling filter
    img = img.resize((img_width, img_height), Image.LANCZOS)
    # Save the resized image to the specified path
    img.save(save_path)

Streaming output truncated to the last 5000 lines.
Resizing image number 2000 / 3499
Resizing image number 2001 / 3499
Resizing image number 2002 / 3499
Resizing image number 2003 / 3499
Resizing image number 2004 / 3499
Resizing image number 2005 / 3499
Resizing image number 2006 / 3499
Resizing image number 2007 / 3499
Resizing image number 2008 / 3499
Resizing image number 2009 / 3499
Resizing image number 2010 / 3499
Resizing image number 2011 / 3499
Resizing image number 2012 / 3499
Resizing image number 2013 / 3499
Resizing image number 2014 / 3499
Resizing image number 2015 / 3499
Resizing image number 2016 / 3499
Resizing image number 2017 / 3499
Resizing image number 2018 / 3499
Resizing image number 2019 / 3499
Resizing image number 2020 / 3499
Resizing image number 2021 / 3499
Resizing image number 2022 / 3499
Resizing image number 2023 / 3499
Resizing image number 2024 / 3499
Resizing image number 2025 / 3499
Resizing image number 2026 / 3499
Resizing image number 2027 / 34

In [ ]:
# Resize validation images
types = ['real', 'fake']
for type_ in types:
  for i, image in enumerate(os.listdir(valid_images_path + type_)):
    print(f"Resizing image number {i} / 1499")
    save_path = Path(resized_valid_images_path + type_ + '_resized' + '/' + image)
    if save_path.is_file():
      continue
    img = Image.open(valid_images_path + type_ + '/' + image)
    img = img.resize((img_width, img_height), Image.LANCZOS)
    img.save(save_path)

KeyboardInterrupt: 

In [ ]:
from keras.applications.inception_v3 import preprocess_input
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # Preprocess images for InceptionV
    rotation_range=40, # Randomly rotate images by up to 40 degrees
    width_shift_range=0.2, # Randomly shift images horizontally by up to 20% of the width
    height_shift_range=0.2, # Randomly shift images vertically by up to 20% of the height
    shear_range=0.2, # Shear images by up to 20 degrees
    zoom_range=0.2, # Randomly zoom images by up to 20%
    horizontal_flip=True, # Randomly flip images horizontally
    fill_mode='nearest', # Fill in newly created pixels with the nearest pixel value
    validation_split=0.2 # Reserve 20% of data for validation
)
# Data augmentation for validation images (same as training for consistency)
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)
# No data augmentation for testing
test_datagen = ImageDataGenerator(
    rescale=1/255
)
# Generate training data
train_generator = train_datagen.flow_from_directory(
    resized_train_images_path,
    target_size=(img_width, img_height),
    shuffle=True,
    seed=20,
    batch_size = batch_size,
    class_mode='binary',
)
# Generate validation data
validation_generator = val_datagen.flow_from_directory(
    resized_valid_images_path,
    target_size=(img_width, img_height),
    seed=20,
    batch_size=batch_size,
    class_mode='binary',
)

Found 7000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [ ]:
# Load the pre-trained InceptionV3 model without the top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width,img_height,3))
# Add custom top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x) # Fully connected layer with 1024 units
predictions = Dense(1, activation='sigmoid')(x) # Output layer for binary classification

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Create the final model
model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
# Train the model using a training data generator
model.fit(
    train_generator,
    batch_size = batch_size, # Define the batch size for training
    epochs=10, # Set the number of epochs to train for
    validation_data=validation_generator,  # Validation data generator for monitoring performance
    callbacks=[EarlyStopping(monitor='val_loss', min_delta=min_delta, patience=patience)] # Callbacks to monitor training process
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


438/438 ━━━━━━━━━━━━━━━━━━━━ 5172s 11s/step - accuracy: 0.6197 - loss: 0.7564 - val_accuracy: 0.6713 - val_loss: 0.6055
Epoch 2/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 185s 380ms/step - accuracy: 0.6717 - loss: 0.6043 - val_accuracy: 0.7030 - val_loss: 0.5771
Epoch 3/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 199s 451ms/step - accuracy: 0.6795 - loss: 0.5923 - val_accuracy: 0.6787 - val_loss: 0.5905
Epoch 4/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 170s 378ms/step - accuracy: 0.6804 - loss: 0.5861 - val_accuracy: 0.6710 - val_loss: 0.6058
Epoch 5/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 168s 379ms/step - accuracy: 0.6915 - loss: 0.5784 - val_accuracy: 0.6980 - val_loss: 0.5735
Epoch 6/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 165s 374ms/step - accuracy: 0.7000 - loss: 0.5742 - val_accuracy: 0.6963 - val_loss: 0.5800
Epoch 7/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 203s 376ms/step - accuracy: 0.7095 - loss: 0.5661 - val_accuracy: 0.7123 - val_loss: 0.5656
Epoch 8/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 167s 377ms/step - accuracy: 0.7178 - loss: 0.553

In [ ]:
# Visualize layers that gonna freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_layer
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_

In [ ]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_layer
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_

In [ ]:
# Freeze the first 249 layers and unfreeze the rest:
for layer in model.layers[:310]:
   layer.trainable = False
for layer in model.layers[310:]:
   layer.trainable = True

In [ ]:
# Import the Stochastic Gradient Descent (SGD) optimizer from Keras optimizers
from keras.optimizers import SGD

# Compile the model by specifying the optimizer, loss function, and metrics
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9),  # Optimizer with learning rate and momentum
              loss=keras.losses.BinaryCrossentropy(),             # Binary crossentropy loss for binary classification
              metrics=['accuracy'])                               # Track accuracy during training

model.fit(
    train_generator,                      # Train data generator
    batch_size = batch_size,              # Number of samples per training update
    epochs=epochs,                        # Number of training iterations
    validation_data=validation_generator, # Validation data generator
    callbacks=[EarlyStopping(monitor='val_loss', min_delta=min_delta, patience=patience)] # Early stopping based on validation loss
)

Epoch 1/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 194s 411ms/step - accuracy: 0.7332 - loss: 0.5355 - val_accuracy: 0.7057 - val_loss: 0.5596
Epoch 2/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 164s 372ms/step - accuracy: 0.7392 - loss: 0.5223 - val_accuracy: 0.7047 - val_loss: 0.5641
Epoch 3/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 205s 380ms/step - accuracy: 0.7416 - loss: 0.5230 - val_accuracy: 0.7170 - val_loss: 0.5485
Epoch 4/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 164s 372ms/step - accuracy: 0.7306 - loss: 0.5337 - val_accuracy: 0.7217 - val_loss: 0.5556
Epoch 5/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 164s 372ms/step - accuracy: 0.7299 - loss: 0.5267 - val_accuracy: 0.7147 - val_loss: 0.5520
Epoch 6/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 203s 374ms/step - accuracy: 0.7404 - loss: 0.5138 - val_accuracy: 0.7127 - val_loss: 0.5515
Epoch 7/50
438/438 ━━━━━━━━━━━━━━━━━━━━ 201s 373ms/step - accuracy: 0.7355 - loss: 0.5232 - val_accuracy: 0.7193 - val_loss: 0.5511


In [ ]:
# Save model weights
model.save_weights('/content/drive/MyDrive/ManipulatedReality/Fine-TunedModel/inceptionv3_tuned_weights_full_22_07.weights.h5')

In [ ]:
# Save model itself
model.save('/content/drive/MyDrive/ManipulatedReality/Fine-TunedModel/inceptionv3_tuned_model_full_22_07.keras')